# Agentic Finance Demo – Autogen AgentChat

This notebook shows how to build a **three‑agent pipeline** that answers liquidity questions using *pre‑computed* metrics for three tickers (`TSLA.US`, `AMZN.US`, `AAPL.US`).

**Agents**
1. **Planner** – rewrites the user question and decides which ticker and time‑scope (“latest” vs “all”).
2. **Executor mini‑team** – `ExecutorWorker` fetches the JSON via simple tools; `Critic` approves.
3. **Financial** – crafts the final answer, computing ratios inline.

We keep the context small because agents read a tidy JSON of ratios, **not raw CSVs**.

## 0 — Install & configure
```bash
pip install -U "autogen-agentchat" "autogen-ext[openai]" pandas python-dotenv
```
Set your OpenAI key **before** running:
```python
OPENAI_API_KEY=...
```

In [ ]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]" pandas python-dotenv

In [ ]:
import warnings, os, json, asyncio, pandas as pd
warnings.filterwarnings('ignore')

In [5]:
API_KEY=''

## 1 — Load pre‑computed metrics & tickers

In [6]:
METRICS = json.load(open('metrics.json'))
SYMBOLS = ["TSLA.US", "AMZN.US", "AAPL.US"]
print(f"Loaded metrics for: {', '.join(METRICS.keys())}")

Loaded metrics for: TSLA.US, AAPL.US, AMZN.US


## 2 — Define two tiny *tools*

In [14]:
import json

# ──  helper that keeps our error-handling consistent  ────────────────
def _symbol_is_valid(symbol: str) -> bool:
    """Return True only if the ticker is in our whitelist *and*
    we actually have metrics stored for it."""
    return symbol in SYMBOLS and symbol in METRICS


def get_latest_metrics(symbol: str) -> str:
    """
    Return a JSON string containing **only the most recent quarter**
    for the requested company.

    Parameters
    ----------
    symbol : str
        One of: "TSLA.US", "AMZN.US", "AAPL.US"

    Returns
    -------
    str
        JSON string of the form:
        { "YYYY-MM-DD": {<metric>: <value>, ...} }
        or "{}" if the symbol is unknown.
    """
    if not _symbol_is_valid(symbol):
        return "{}"

    # Pick the chronologically-last date key.
    latest_date = max(METRICS[symbol])          # dates are "YYYY-MM-DD"
    latest_data = METRICS[symbol][latest_date]

    return json.dumps({latest_date: latest_data})


def get_all_metrics(symbol: str) -> str:
    """
    Return **every stored quarter** for the symbol as one JSON string.

    Same return contract as `get_latest_metrics`, but the dictionary
    can contain many date keys.
    """
    if not _symbol_is_valid(symbol):
        return "{}"

    return json.dumps(METRICS[symbol])
# ── register functions as LLM-callable tools  ───────────────────────
TOOLS = [get_latest_metrics, get_all_metrics]

# quick sanity check
print(get_latest_metrics("TSLA.US")[:120] + "  …")

{"2025-03-31": {"current_ratio": 2.0, "quick_ratio": 1.54, "cash_ratio": 1.24, "debt_to_equity": 0.67, "free_cash_flow":  …


## 3 — Shared OpenAI client

In [15]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
client = OpenAIChatCompletionClient(model='gpt-4o-mini', temperature=0,api_key=API_KEY)

## 4 — Planner agent

In [16]:
from autogen_agentchat.agents import AssistantAgent

PLANNER_SYS = f"""
Rewrite the user question briefly.
Pick ONE ticker from: {', '.join(SYMBOLS)}.
Return JSON ONLY: {{'refined':'...','symbol':'<ticker>','scope':'latest'|'all'}}
"""
Planner = AssistantAgent('Planner', system_message=PLANNER_SYS, model_client=client)

## 5 — Executor mini‑team

In [17]:
EXECUTOR_SYS = """
Input = Planner JSON. If scope=='latest' call latest_metrics(symbol) else metrics_all_dates(symbol).
Wrap as { 'symbol':symbol, 'data':<json> } then print json.dumps(...) + APPROVE.
"""
ExecutorWorker = AssistantAgent('ExecutorWorker', system_message=EXECUTOR_SYS, model_client=client, tools=TOOLS, reflect_on_tool_use=True)

CRITIC_SYS = "Reply APPROVE if JSON includes data, otherwise describe what is missing."
Critic = AssistantAgent('Critic', system_message=CRITIC_SYS, model_client=client)

from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat

termination = TextMentionTermination('APPROVE') | MaxMessageTermination(12)
executor_chat = RoundRobinGroupChat([ExecutorWorker, Critic], termination_condition=termination)

## 6 — Financial agent

In [18]:
FIN_SYS = """
You will get user_question, refined, data_json. ### Continue Prompt here ###
"""
Financial = AssistantAgent('Financial', system_message=FIN_SYS, model_client=client)

## 7 — Run the full chain

In [23]:
from autogen_agentchat.ui import Console

async def answer(q:str):
    print('🟡 USER:', q)
    plan_res = await Console(Planner.run_stream(task=q))
    plan = plan_res.messages[-1].content
    print('\n🟠 PLANNER:', plan)
    exec_res = await Console(executor_chat.run_stream(task=plan))
    data_msg = [m for m in exec_res.messages if m.source=='ExecutorWorker' and m.type=='TextMessage'][-1]
    data = data_msg.content
    print('\n🟢 EXECUTOR (truncated):', data[:120]+' …')
    fin = await Console(Financial.run_stream(task=f'user_question:{q}\nrefined:{plan}\ndata_json:{data}'))
    print('\n🔵 ANSWER:\n', fin.messages[-1].content)

await answer("Based on the most recent filings, is Tesla's liquidity position improving?")

🟡 USER: Based on the most recent filings, is Tesla's liquidity position improving?
---------- TextMessage (user) ----------
Based on the most recent filings, is Tesla's liquidity position improving?
---------- TextMessage (Planner) ----------
{'refined':'Is Tesla's liquidity position improving based on recent filings?','symbol':'TSLA.US','scope':'latest'}

🟠 PLANNER: {'refined':'Is Tesla's liquidity position improving based on recent filings?','symbol':'TSLA.US','scope':'latest'}
---------- TextMessage (user) ----------
{'refined':'Is Tesla's liquidity position improving based on recent filings?','symbol':'TSLA.US','scope':'latest'}
---------- ToolCallRequestEvent (ExecutorWorker) ----------
[FunctionCall(id='call_Hiw0C6FGId8dufh31aZNpoPQ', arguments='{"symbol":"TSLA.US"}', name='get_latest_metrics')]
---------- ToolCallExecutionEvent (ExecutorWorker) ----------
[FunctionExecutionResult(content='{"2025-03-31": {"current_ratio": 2.0, "quick_ratio": 1.54, "cash_ratio": 1.24, "debt_to_equ